In [1]:
import pandas as pd
import numpy as np
import requests
import time
from Bio import motifs
from Bio.motifs import jaspar
from Bio.Seq import Seq

In [2]:
# rename columns
def remove_rename(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(["peak.1", "gene.1"], axis=1)

    num_cells = (len(df.columns)-4) // 2
    new_cols = ['peak', 'gene', 'Pair', 'is_pair'] + ["atac." + str(x) for x in range(num_cells)] + ["rna." + str(x) for x in range(num_cells)]
    df = df.rename(columns=dict(zip(df.columns, new_cols)))
    return df

def set_labels(df: pd.DataFrame) -> pd.DataFrame:
    df["is_pair"] = df["is_pair"].apply(lambda x: 1 if x else 0)
    return df
    
def normalize(df: pd.DataFrame) -> pd.DataFrame:
    # only counts
    num_df = df[df.columns[4:]]
    
    # Summing each column (sample) to get library sizes
    library_sizes = num_df.sum(axis=0)
    
    # Normalizing to CPM
    cpm_df = num_df.div(library_sizes, axis=1) * 10**6
    
    # If you want to log-transform the CPM data
    log_cpm_df = cpm_df.apply(lambda x: np.log2(x + 1))

    return pd.concat([df[df.columns[:4]], log_cpm_df], axis=1)
    
DATA_PATH = "../data/Tab_delimited_text/"
train_df = pd.read_csv(DATA_PATH + "train.csv")
test_df = pd.read_csv(DATA_PATH + "test.csv")

train_df = remove_rename(train_df)
train_df = set_labels(train_df)
train_df = normalize(train_df)

test_df = remove_rename(test_df)
test_df = set_labels(test_df)
test_df = normalize(test_df)

In [3]:
train_df.head()

,peak,gene,Pair,is_pair,atac.0,atac.1,atac.2,atac.3,atac.4,atac.5,...,rna.2989,rna.2990,rna.2991,rna.2992,rna.2993,rna.2994,rna.2995,rna.2996,rna.2997,rna.2998
0,chr1-89196985-89201657,GBP2,chr1-89196985-89201657_GBP2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,11.752078,12.693382,0.000000,13.073755,11.679343,13.550867,12.522423,11.77211,13.693273,0.0
1,chr6-33077557-33083333,HLA-DPA1,chr6-33077557-33083333_HLA-DPA1,1,0.000000,13.143825,11.839205,0.000000,13.313733,0.000000,...,0.000000,0.000000,12.295231,12.073922,11.679343,11.966145,0.000000,11.77211,13.108365,0.0
2,chr6-137789753-137792920,TNFAIP3,chr6-137789753-137792920_TNFAIP3,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,10.710842,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
3,chr1-212604203-212626574,ATF3,chr1-212604203-212626574_ATF3,1,14.073671,0.000000,13.423905,13.170174,11.144304,15.514953,...,11.752078,12.693382,13.517459,0.000000,12.679123,0.000000,12.522423,0.00000,13.108365,0.0
4,chr2-96541661-96555628,ARID5A,chr2-96541661-96555628_ARID5A,1,0.000000,0.000000,12.839008,0.000000,13.728735,12.707783,...,0.000000,0.000000,10.710842,12.073922,11.679343,0.000000,0.000000,0.00000,0.000000,0.0


In [4]:
def add_correlation(df: pd.DataFrame) -> pd.DataFrame:
    def correl(row):
        atac_counts = row.filter(like="atac.").values.astype(float)
        rna_counts = row.filter(like="rna.").values.astype(float)
        return np.corrcoef(atac_counts, rna_counts)[0, 1]

    def covar(row):
        atac_counts = row.filter(like="atac.").values.astype(float)
        rna_counts = row.filter(like="rna.").values.astype(float)
        return np.cov(atac_counts, rna_counts)[0, 1]
    
    df["correlation"] = df.apply(correl, axis=1)
    df["covariance"] = df.apply(covar, axis=1)
    return df

manual_synonyms = {
    "C18orf25": "ARK2N",
    "C1orf112": "FIRRM",
    "C12orf65": "MTRFR",
    "VNN3": "VNN3P",
}

def get_gene_location(gene_name: str):
    # apply manual synonyms
    if gene_name in list(manual_synonyms.keys()):
        gene_name = manual_synonyms[gene_name]

    server = "https://rest.ensembl.org"
    ext = f"/lookup/symbol/homo_sapiens/{gene_name}?expand=1"
    headers = {"Content-Type": "application/json"}

    response = requests.get(server + ext, headers=headers)

    if not response.ok:
        print("Gene " + gene_name + " was not found in Ensembl.")
        return None

    gene_data = response.json()
    gene_location = {
        "chromosome": gene_data["seq_region_name"],
        "start": int(gene_data["start"]),
        "end": int(gene_data["end"]),
    }
    time.sleep(0.05)
    return gene_location

def get_peak_location(peak_str: str):
    splits = peak_str.split("-")
    return {
        "chromosome": splits[0].replace("chr", ""),
        "start": int(splits[1]),
        "end": int(splits[2]),
    }

def get_peak_sequence(peak_str: str):
    peak_loc = get_peak_location(peak_str)
    
    server = "https://rest.ensembl.org"
    ext = f"/sequence/region/human/{peak_loc['chromosome']}:{peak_loc['start']}..{peak_loc['end']}:1?"
    headers = {"Content-Type": "text/plain"}

    response = requests.get(server + ext, headers=headers)

    if not response.ok:
        print("Peak @ " + peak_str + " was not found in Ensembl.")
        return None

    time.sleep(0.05)
    return response.text

def add_distance(df: pd.DataFrame) -> pd.DataFrame:
    def dist(row):
        gene_pos = get_gene_location(row["gene"])
        peak_pos = get_peak_location(row["peak"])
        
        if gene_pos == None:
            return None
            
        if gene_pos["chromosome"] != peak_pos["chromosome"]:
            print(f"not on same chromosome: peak ({peak_pos['chromosome']}) | gene ({gene_pos['chromosome']})")
            return None

        dist1 = abs(gene_pos["start"] - peak_pos["end"])
        dist2 = abs(gene_pos["end"] - peak_pos["start"])
        return min(dist1, dist2)

    df["distance"] = df.apply(dist, axis=1)
    return df

def add_peak_seq(df: pd.DataFrame) -> pd.DataFrame:
    def seq(row):
        return get_peak_sequence(row["peak"])

    df["peak_seq"] = df.apply(seq, axis=1)
    return df

In [ ]:
feature_df = add_correlation(train_df)
feature_df = add_distance(feature_df)
feature_df = add_peak_seq(feature_df)

In [5]:
feature_df_test = add_correlation(test_df)
feature_df_test = add_distance(feature_df_test)
feature_df_test = add_peak_seq(feature_df_test)

In [7]:
feature_df[["Pair", "correlation", "distance", "peak_seq", "is_pair"]]

,Pair,correlation,distance,peak_seq,is_pair
0,chr1-89196985-89201657_GBP2,0.102802,46529,AGAATTAGAAGTTTTCTTCATTGGGAGAAGTCATGCCCTTCTTTAG...,1
1,chr6-33077557-33083333_HLA-DPA1,0.089582,3218,AGGTGCTGGAGAGGATGTGGAGAAATAGGAACACTTTTACTCTGTT...,1
2,chr6-137789753-137792920_TNFAIP3,0.079216,74294,GCTCAAAAAAAAAAAAAAAAAGGAATAAACTACAAGTTTATGCAAC...,1
3,chr1-212604203-212626574_ATF3,0.149961,16574,CTGAGACATCGTGTTTATGACTTGTATACTCACTCAGTTGACTGGA...,1
4,chr2-96541661-96555628_ARID5A,0.057503,10973,ACAAGTGAGTAATGAGACAATAATCACCATCATCATCATTATTGCT...,1
...,...,...,...,...,...
295,chr1-151593645-151594727_SNX27,-0.001384,17279,TTGACAGATGAGTAAACCGAGGCCCAATGGCGGGGGTGGGGGAGTG...,0
296,chr14-52692182-52692881_PSMC6,0.025952,14297,CACCAGGCTCACTTCTTGGAAGTGTCACACCTTTAATTCATTTTTT...,0
297,chr19-45527935-45529784_FOSB,0.001425,52756,GACAGGTAGTCCCCCTGGAGGAGGCCTTTGCGTGAAGGAGGGCGGA...,0
298,chr17-32352719-32356100_ZNF207,0.008542,5968,ATGGCCTTGCTCTGTGGCCCAGGCTGGATTGCAGTGACATGGTTAT...,0


In [12]:
# Load the JASPAR PFM file
with open("../data/JASPAR2024_CORE_non-redundant_pfms_jaspar.txt") as handle:
    records = jaspar.read(handle, "jaspar")

motifs = []
for motif in records:
    pwm = motif.counts.normalize(pseudocounts=0.0001)  # Normalize with pseudocounts
            
    # Calculate scores
    mat = pwm.log_odds({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})

    motifs.append(mat)

def score_tf_motif(seq, in_motif_agg="sum", out_motif_agg="mean"):
    sequence = Seq(seq)
    motif_scores = []
    for motif in motifs:
        scores = motif.calculate(sequence)

        if in_motif_agg == "sum":
            motif_scores.append(np.sum(scores))
        elif in_motif_agg == "max":
            motif_scores.append(np.max(scores))

    if out_motif_agg == "mean":
        return np.mean(motif_scores)
    elif out_motif_agg == "max":
        return np.max(motif_scores)

def raw_scores(df):
    raw_scores = []
    for _, row in df.iterrows():
        sequence = Seq(row["peak_seq"])
        motif_scores = []
        for motif in motifs:
            scores = motif.calculate(sequence)
            motif_scores.append(np.sum(scores))

        raw_scores.append(tuple(motif_scores))

    return pd.DataFrame(raw_scores, columns=["motif_" + str(x) for x in range(len(motifs))])
    
def add_tf_scores(df, in_agg, out_agg):
    def score(row):
        return score_tf_motif(row["peak_seq"], in_agg, out_agg)

    df["score_" + in_agg + "_" + out_agg] = df.apply(score, axis=1)
    return df

def add_raw_scores(df):
    raw_df = raw_scores(df)
    return pd.concat([df, raw_df], axis=1)

In [11]:
feature_df = add_tf_scores(feature_df)

In [8]:
feature_df_test = add_tf_scores(feature_df_test)

In [12]:
feature_df.to_csv("../data/train_features.csv", index=False)
feature_df.head()

,peak,gene,Pair,is_pair,atac.0,atac.1,atac.2,atac.3,atac.4,atac.5,...,rna.2994,rna.2995,rna.2996,rna.2997,rna.2998,correlation,covariance,distance,peak_seq,peak_tf_score
0,chr1-89196985-89201657,GBP2,chr1-89196985-89201657_GBP2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.550867,12.522423,11.77211,13.693273,0.0,0.102802,2.703429,46529,AGAATTAGAAGTTTTCTTCATTGGGAGAAGTCATGCCCTTCTTTAG...,-2.178348e+05
1,chr6-33077557-33083333,HLA-DPA1,chr6-33077557-33083333_HLA-DPA1,1,0.000000,13.143825,11.839205,0.000000,13.313733,0.000000,...,11.966145,0.000000,11.77211,13.108365,0.0,0.089582,3.225886,3218,AGGTGCTGGAGAGGATGTGGAGAAATAGGAACACTTTTACTCTGTT...,-2.721785e+05
2,chr6-137789753-137792920,TNFAIP3,chr6-137789753-137792920_TNFAIP3,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.0,0.079216,2.646761,74294,GCTCAAAAAAAAAAAAAAAAAGGAATAAACTACAAGTTTATGCAAC...,-1.476451e+05
3,chr1-212604203-212626574,ATF3,chr1-212604203-212626574_ATF3,1,14.073671,0.000000,13.423905,13.170174,11.144304,15.514953,...,0.000000,12.522423,0.00000,13.108365,0.0,0.149961,4.705232,16574,CTGAGACATCGTGTTTATGACTTGTATACTCACTCAGTTGACTGGA...,-1.058393e+06
4,chr2-96541661-96555628,ARID5A,chr2-96541661-96555628_ARID5A,1,0.000000,0.000000,12.839008,0.000000,13.728735,12.707783,...,0.000000,0.000000,0.00000,0.000000,0.0,0.057503,1.837850,10973,ACAAGTGAGTAATGAGACAATAATCACCATCATCATCATTATTGCT...,-6.657365e+05


In [9]:
feature_df_test.to_csv("../data/test_features.csv", index=False)
feature_df_test.head()

,peak,gene,Pair,is_pair,atac.0,atac.1,atac.2,atac.3,atac.4,atac.5,...,rna.2994,rna.2995,rna.2996,rna.2997,rna.2998,correlation,covariance,distance,peak_seq,peak_tf_score
0,chr1-1245493-1248050,SDF4,chr1-1245493-1248050_SDF4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,11.727423,0.000000,0.0,0.003734,0.042858,13462,CGGCACCCTTTGAGGGAGGCCCGCCACCCTGCAGGGTCTCTGGAGG...,-1.236289e+05
1,chr1-1330394-1334148,MRPL20,chr1-1330394-1334148_MRPL20,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,12.324520,0.000000,11.231729,0.0,0.005909,0.104767,67761,TGAGTGCCCCCCAGTTCCCCTGGGAGGGCCTGCGCCTGGAGTCTGC...,-1.818039e+05
2,chr1-2145904-2147150,FAAP20,chr1-2145904-2147150_FAAP20,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,-0.001309,-0.015141,37311,CTGTGACCCTGTCTGTAAAAAAAAGTTCTTAATTTTAAAAAGTCAC...,-5.889110e+04
3,chr1-9713011-9736481,PIK3CD,chr1-9713011-9736481_PIK3CD,1,13.812728,13.400321,14.186812,13.124375,13.304177,0.000000,...,0.0,13.324379,14.312031,11.231729,0.0,0.054654,2.350537,16103,ATCTCTACTGAAAATACAAAAAATTAGCCAGGTGTGGTGGTAGATG...,-1.121144e+06
4,chr1-21287896-21301043,ECE1,chr1-21287896-21301043_ECE1,1,0.000000,13.400321,13.771800,0.000000,14.304106,12.914947,...,0.0,0.000000,0.000000,12.231429,0.0,0.067243,2.316125,57676,TCCTTTTTATACTTGACTATCTTTTTTTCTCCCCCAAGTGAGCATA...,-6.239588e+05


# add various tf scores

In [13]:
train_df = pd.read_csv("../data/train_features.csv")
test_df = pd.read_csv("../data/test_features.csv")

In [3]:
train_df = train_df.drop(columns=["peak_tf_score"])
test_df = test_df.drop(columns=["peak_tf_score"])

In [23]:
train_df = add_tf_scores(train_df, "max", "max")

In [24]:
train_df.score_max_max.describe()

count    300.000000
mean      23.300882
std        5.852078
min       16.136665
25%       19.591942
50%       21.965355
75%       25.883785
max       54.213387
Name: score_max_max, dtype: float64

In [27]:
train_df.to_csv("../data/train_features_2.csv", index=False)

In [14]:
train_df = add_raw_scores(train_df)
test_df = add_raw_scores(test_df)

In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [19]:
motif_cols = ["motif_" + str(x) for x in range(len(motifs))]
def normalize_motif_cols(df):
    scaler = StandardScaler()
    for col in motif_cols:
        df[col] = scaler.fit_transform(df[[col]])

    return df

def add_pca_components(df, n_components):
    pca = PCA(n_components=n_components)
    pca_components = pca.fit_transform(df[motif_cols])

    # Create new column names for the principal components
    component_names = [f'pc_{i+1}' for i in range(n_components)]
    
    # Create a DataFrame with the PCA components
    pca_df = pd.DataFrame(data=pca_components, columns=component_names)
    
    # Concatenate the original DataFrame with the PCA components
    df_with_pca = pd.concat([df, pca_df], axis=1)
    
    return df_with_pca

In [21]:
train_pca_df = normalize_motif_cols(train_df)
train_pca_df = add_pca_components(train_pca_df, n_components=20)

test_pca_df = normalize_motif_cols(test_df)
test_pca_df = add_pca_components(test_pca_df, n_components=20)

In [23]:
train_pca_df.to_csv("../data/train_features_3.csv", index=False)
test_pca_df.to_csv("../data/test_features_3.csv", index=False)